In [2]:
# %load examples/basic_sqlite.py
"""
Simple poll app
"""
# built-in
from datetime import datetime
from pprint import pprint
from contextlib import ExitStack

# 3rd party
import matplotlib.pyplot as plt

# core
from dorm.database.drivers.sqlite import Sqlite
from dorm.database.drivers.mysql import Mysql

In [3]:
# %load config.py
CLUSTERS = [
    {'address':'127.0.0.1', 'name':'local_cluster_1', 'port':'40001'},
    {'address':'127.0.0.1', 'name':'local_cluster_2', 'port':'40002'},
    {'address':'127.0.0.1', 'name':'local_cluster_3', 'port':'40003'},
    {'address':'127.0.0.1', 'name':'local_cluster_4', 'port':'40004'},
]

DATABASES = {
    'sqlite': [
        {'database_name': 'datastore/poll_1.db'},
        {'database_name': 'datastore/poll_2.db'},
        {'database_name': 'datastore/poll_3.db'},
        {'database_name': 'datastore/poll_4.db'},
        {'database_name': 'datastore/poll_5.db'},
        {'database_name': 'datastore/poll_6.db'},
        {'database_name': 'datastore/poll_7.db'},
        {'database_name': 'datastore/poll_8.db'},
        {'database_name': 'datastore/poll_9.db'},
        {'database_name': 'datastore/poll_10.db'}
    ],
    'postgres': [
        {'database_name': 'test',
         'user': 'postgres',
         'password':'mysecretpassword',
         'host':'127.0.0.1',
         'port': 5432
        },
    ],
    'mysql': [
        {'database_name': 'employees',
         'user': 'root',
         'password':'mysecretpassword',
         'host':'127.0.0.1',
         'port': 3306
        },
    ]
}

# sudo docker run --name dorm_test_mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=mysecretpassword -d mysql
# sudo docker run --name dorm_postgres -p 5432:5432  -e  POSTGRES_PASSWORD=mysecretpassword -d postgres


## SQLite

In [4]:
db = Sqlite(DATABASES['sqlite'][0])

In [5]:
def get_table_structure(tq):
    tables = []
    q = "SELECT sql FROM sqlite_master;"
    tqs = db.execute(q).fetchall()
    for tq in tqs:
        t = tq[0].replace("CREATE TABLE ", "")
        table_name = t[:t.find(" ")]
        columns = t[t.find(" "):].strip()[1:-1].split(", ")
        tables.append({table_name:columns})
    return tables

In [6]:
tdl = get_table_structure(db)

'SELECT sql FROM sqlite_master;'


In [7]:
tdl

[{'question': ['id INTEGER NOT NULL PRIMARY KEY',
   'question_text CHAR(200)',
   'pub_date TIMESTAMP']},
 {'choice': ['votes INTEGER',
   'id INTEGER NOT NULL PRIMARY KEY',
   'question INTEGER NOT NULL REFERENCES question (id)',
   'choice_text CHAR(200)']},
 {'testmodel': ['"id" INTEGER NOT NULL PRIMARY KEY',
   '"test_field" CHAR(200)']}]

In [15]:
def build_model():
    class_str = """class {}(models.Model):\n"""
    field_str = """    {} = models.{}({})\n"""
    code = ""
    for structure in get_table_structure(db):
        model=""
        for key, val in structure.items():

            model = class_str.format(key.title())
            for column in val:
                extra = []
                column = column.lower()
                #extra.append("null=False")
                #extra.append("unique=False")

                field_name, field_type, *_ = column.split(" ")
                if "(" in field_type:
                    extra.append("max_length="+field_type[field_type.find("(")+1:field_type.find(")")])
                    field_type = field_type[:field_type.find("(")]
                if "primary key" in column:
                    model += field_str.format(field_name, "PrimaryKey", ", ".join(extra))
                elif "references" in column:
                    target_table = column[:column.find("REFERENCES ")].split(" (")[0].split(" ")[0]
                    extra.append(target_table.title())
                    model += field_str.format(field_name, "ForeignKey", ", ".join(extra))
                else:
                    model += field_str.format(field_name, field_type.title(), ", ".join(extra))
        code +=model
    return code

In [16]:
print(build_model())

'SELECT sql FROM sqlite_master;'
class Question(models.Model):
    id = models.PrimaryKey()
    question_text = models.Char(max_length=200)
    pub_date = models.Timestamp()
class Choice(models.Model):
    votes = models.Integer()
    id = models.PrimaryKey()
    question = models.ForeignKey(Question)
    choice_text = models.Char(max_length=200)
class Testmodel(models.Model):
    "id" = models.PrimaryKey()
    "test_field" = models.Char(max_length=200)



In [9]:
print(ms)

class Question(models.Model):
    id = models.PrimaryKey()
    question_text = models.Char(max_length=200)
    pub_date = models.Timestamp()

class Choice(models.Model):
    votes = models.Integer()
    id = models.PrimaryKey()
    question = models.ForeignKey(Question)
    choice_text = models.Char(max_length=200)



In [10]:
from dorm.database import models

In [11]:
exec(ms)

In [12]:
Choice.select().all()

'select * from choice;'
'select * from choice;'


<generator object SelectQuery._execute.<locals>.<genexpr> at 0x7fcfe3aabbf8>

In [13]:
q1 = Question.select().first()

'select * from question limit 1;'
'select * from question limit 1;'


In [22]:
r1 = q1.update(id=1).set(question_text="How are you?")

In [23]:
r1.sql

'update question set question_text="How are you?" where id="1"'